In [2]:
import os
import zipfile
import subprocess
import shutil

# Step 1: Set Kaggle API Key location
kaggle_json_path = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_json_path, exist_ok=True)

# NOTE: Adjust this if your kaggle.json is in the notebook folder
local_kaggle_json = "kaggle.json"

if not os.path.exists(local_kaggle_json):
    raise FileNotFoundError("⚠️ kaggle.json not found in the notebook directory!")

# Move kaggle.json to ~/.kaggle and set permissions
shutil.copy(local_kaggle_json, os.path.join(kaggle_json_path, "kaggle.json"))
os.chmod(os.path.join(kaggle_json_path, "kaggle.json"), 0o600)

# Step 2: Create data directory
data_path = "../data/breakhis"  # Going one level up from notebooks/
os.makedirs(data_path, exist_ok=True)

# Step 3: Run Kaggle CLI to download and unzip
subprocess.run([
    "kaggle", "datasets", "download",
    "-d", "ambarish/breakhis",
    "-p", data_path,
    "--unzip"
], check=True)


Dataset URL: https://www.kaggle.com/datasets/ambarish/breakhis
License(s): unknown


100%|██████████| 3.99G/3.99G [00:04<00:00, 950MB/s] 


CompletedProcess(args=['kaggle', 'datasets', 'download', '-d', 'ambarish/breakhis', '-p', '../data/breakhis', '--unzip'], returncode=0)